Attempt to plot velocity-integrated maps of $^{12}\mathrm{C}^{16}\mathrm{O}$ J(1-0) emission

* ThrUMMS - http://www.astro.ufl.edu/~peterb/research/thrumms/rbank/ (Barnes+ 2015)
* NANTEN - Saito+ 2001 (http://adsabs.harvard.edu/abs/2001PASJ...53.1037S but, no data access?...)
* Dame+ 2001, DHT36 (low resolution)
* MOPRA CO survey - mopra.org (not public)

Possible software packages of use:
* yt http://yt-project.org/doc/cookbook/fits_radio_cubes.html
* spectral-cube http://radio-astro-tools.github.io/

In [ ]:
%pylab
%matplotlib inline

import aplpy
from astropy.io import fits
import astropy.units as u
import numpy as np
import os
from spectral_cube import SpectralCube

## First look integration of ThrUMMS cubes

Based on Saito+ 2001 Fig. 1a, start by integrating -64 km/s to -34 km/s.
Quick reference: https://casa.nrao.edu/docs/CasaRef/image.moments.html

In [ ]:
def view_and_dump_cubes(filename, low=-64, high=-34, look=True, write=False, **kwargs):
    """Load a cube, view moment0 map, then integrate over -64 to -34 km/s (Cen arm velocities)"""
    cube = SpectralCube.read(filename)
    print cube
    #moment0 = cube.moment0()
    #if look:
    #    moment0.quicklook()

    subcube = cube.spectral_slab(low * u.km / u.s, high * u.km / u.s)
    submoment0 = subcube.moment0()
    if look:
        submoment0.quicklook()
    
    if write:
        #moment0.write('data_intgr/moment0_' + os.path.basename(filename), **kwargs)
        submoment0.write('data_intgr/submoment0_{:+03d}_to_{:+03d}_{:s}'.format(low, high, os.path.basename(filename)),
                         **kwargs)

ThrUMMS 12CO data from NAOJ website

In [ ]:
view_and_dump_cubes('data/Th_IV_309.75_12CO.fits', write=True, overwrite=True)

ThrUMMS 12CO data from UFL website

In [ ]:
view_and_dump_cubes('data/dr3.s306.12co.fits', write=True, overwrite=True)

CfA survey (DHT36 area) 12CO interpolated @ high latitude

Latest stable pip install of spectral_cube (0.4.0) has a bug in header keyword handling,
where it attempts to delete ''-keyed header entries but fails.
This was patched 2017 Jan. 12 with commit 99a2e7c4ffe3d30276c87d8f9efc6f273a6defb5.
Therefore I install the latest dev version of the code.

In [ ]:
cube = SpectralCube.read('data/DHT36_Quad4_interp.fits')
print cube
subcube = cube.spectral_slab(-64 * u.m / u.s, -34 * u.m / u.s)  # Hack; parsed vLSR units are incorrect
cube.moment0().quicklook()
subcube.moment0().quicklook()

cube.moment0().write('data_intgr/moment0_DHT36_Quad4_interp.fits', overwrite=True)
subcube.moment0().write('data_intgr/submoment0_-64_to_-34_DHT36_Quad4_interp.fits', overwrite=True)

## Integrate over hand-chosen velocity ranges...

In [ ]:
view_and_dump_cubes('data/Th_IV_309.75_12CO.fits', low=-70, high=0, look=True, write=True, overwrite=True)
view_and_dump_cubes('data/Th_IV_309.75_12CO.fits', low=-70, high=+70, look=True, write=True, overwrite=True)
# And partition, too
view_and_dump_cubes('data/Th_IV_309.75_12CO.fits', low=-70, high=-35, look=True, write=True, overwrite=True)
view_and_dump_cubes('data/Th_IV_309.75_12CO.fits', low=-35, high=0, look=True, write=True, overwrite=True)
view_and_dump_cubes('data/Th_IV_309.75_12CO.fits', low=0, high=+70, look=True, write=True, overwrite=True)

## Generate velocity-binned slices

In [ ]:
for f, unit in zip(['DHT36_Quad4_interp.fits', 'Th_IV_309.75_12CO.fits'],
                   [u.m/u.s, u.km/u.s]):  # really silly hack for DHT file

    cube = SpectralCube.read('data/' + f)

    bins = [(x, x+5) for x in range(-80, 80, 5)]
    for a, b in bins:
        subcube = cube.spectral_slab(a * unit, b * unit)
        subcube.moment0().write('data_intgr/submoment0_{:+03d}_to_{:+03d}_{:s}'.format(a,b,f), overwrite=True)
        print "Wrote {:+03d} to {:+03d} slice of {}".format(a,b,f)

Colorbars and multi-panel plots in APLPy are a bit funky.  See:
* https://github.com/aplpy/aplpy/issues/119
* https://github.com/aplpy/aplpy/issues/174

Main issues:
1. aspect ratio preservation means weird sizing control via figsize, subplot_params.  hspace and wspace cannot always be respected.
2. inline vs. file-writing backends place objects/labels differently

In [ ]:
fig = plt.figure(figsize=(7, 1.75))

most = fits.open('../most/G309.2-0.6.fits')
cmap = 'afmhot_r'
#cmap = 'cubehelix_r'

#bins = [(x, x+5) for x in range(-50, -30, 5)]
bins = [(-70,70), (-70,-35), (-35,0), (0,+70)]
nrow = 1
ncol = 4
assert nrow*ncol == len(bins)

for n, intv in enumerate(bins):
    
    print "Subplot #{} ({:g} to {:g} km/s)".format(n+1, *intv)
    #f = aplpy.FITSFigure('data_intgr/submoment0_{:+03d}_to_{:+03d}_DHT36_Quad4_interp.fits'.format(*intv),
    f = aplpy.FITSFigure('data_intgr/submoment0_{:+03d}_to_{:+03d}_Th_IV_309.75_12CO.fits'.format(*intv),
                         figure=fig, subplot=(nrow, ncol, n+1))
    f.recenter(309.2, -0.6, width=0.8, height=0.8)
    #f.show_colorscale(vmin=0, vmax=50, stretch='linear',cmap=cmap)
    f.show_colorscale(vmin=0, vmax=100000, stretch='linear',cmap=cmap)
    
    f.add_label(0.1, 0.85, '[{:g},{:g}] km/s'.format(*intv),
                relative=True, horizontalalignment='left',
                size=8, color='black')
    f.show_contour(data=most, levels=np.logspace(-2, -0.5, 4),  # Sparse log contours, 0.01 to 0.1*sqrt(10)
                   colors='cyan', linewidths=0.75)
    #f.show_regions('../xmm/regs/lobe_ne.reg')
    #f.show_regions('../xmm/regs/lobe_sw.reg')
    f.tick_labels.hide()
    f.axis_labels.hide()
    
    f.tick_labels.set_font(size=8)
    f.axis_labels.set_font(size=8)

    f.ticks.set_color('black')
    f.ticks.set_yspacing(0.5)
    f.ticks.set_minor_frequency(5)
    f.ticks.set_xspacing(0.5)
    f.ticks.set_minor_frequency(5)
    
    f.ticks._ax1.tick_params(axis='both', which='both', direction='in')
    f.ticks._ax2.tick_params(axis='both', which='both', direction='in')
    
    if n % ncol == 0:  # Left row
        f.tick_labels.set_yformat('dd.d')
        f.tick_labels.show_y()
        f.axis_labels.show_y()
    
    if n >= len(bins) - ncol:  # Bottom row
        f.tick_labels.set_xformat('ddd.d')
        f.tick_labels.show_x()
        f.axis_labels.show_x()
    
    if n == len(bins)-1:  # Last plot
        f.add_colorbar(pad=0, width=0)  # setting pad=0,width=0 prevents f from resizing
        
        # inline plot
        #f.colorbar.set_box([0.96, 0.025, 0.03, 0.95])  # If plot is taller than desired for grid
        #f.colorbar.set_box([0.96, 0, 0.03, 1])  # If plot is wider than desired for grid
        
        # savefig dump
        f.colorbar.set_box([0.88, 0.075, 0.02, 0.85])  # savefig dump
        
        f.colorbar.set_axis_label_text('K m/s')  # TODO want to rescale images dimensions
        f.colorbar._colorbar_axes.tick_params(axis='both', which='both', direction='in')
        f.colorbar.set_axis_label_font(size=8)
        f.colorbar.set_font(size=8)
        #f.colorbar.set_axis_label_rotation(90)

# inline plot
#fig.subplots_adjust(left=0, bottom=0, right=0.94, top=1, wspace=0.05, hspace=0.05)
#fig.subplots_adjust(left=0, bottom=0, right=0.94, top=1, wspace=0.05, hspace=0.05)

# For savefig dump
fig.subplots_adjust(left=0.075, bottom=0.225, right=0.88, top=0.925, wspace=0, hspace=0)

fig.canvas.draw()
fig.savefig('fig_co_snapshots.pdf')

## Plot galactic longitude l vs. v_LSR

In [ ]:
#cube = SpectralCube.read('data/Th_IV_309.75_12CO.fits')
cube = SpectralCube.read('data/DHT36_Quad4_interp.fits')

In [ ]:
cube.moment(order=0, axis=1).quicklook()  # This one doesn't seem to work on the Thrumms data set, inexplicably